In [1]:
from numpy import pi, linspace, diff, mean, min, max, arctan2
from skyfield.api import load
from skyfield.framelib import ecliptic_J2000_frame, ICRS, mean_equator_and_equinox_of_date
from skyfield.searchlib import find_discrete
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [2]:
output_notebook()

Loading BokehJS ...

In [9]:
ts = load.timescale()
eph_s_name = 'de421.bsp'
# eph_s_name = 'de422.bsp'
t_start = ts.tt(2005, 6, 21, 0, 0, 0)
eph_s = load(eph_s_name)
moon = eph_s["moon"]
earth = eph_s["earth"]
sun = eph_s["sun"]
northern_hemisphere = True

def get_colors(tt):
    return [f"#{255:02x}{int(255 * (tt[-1] - t) / (tt[-1] - tt[0])):02x}{255:02x}" for t in tt]

## Max lunar declination

In [4]:
def f_max_dec(target, ref):
    def zero(t):
        _, _, _, lat_r, _, _  = ref.at(t).observe(target).frame_latlon_and_rates(ICRS)
        return  lat_r.degrees.per_day > 0
    zero.step_days = 10.0
    return zero

In [14]:
t_dec_zero_rate = find_discrete(t_start, t_start + 365*19.5, f_max_dec(moon, earth))
t_max_dec = t_dec_zero_rate[0][t_dec_zero_rate[1] == int(not northern_hemisphere)]
ra, dec, dist = earth.at(t_max_dec).observe(moon).radec()

In [15]:
colors = get_colors(t_max_dec.tt)
fig = figure(title="Moon standstills", x_axis_label="RA(hours)", y_axis_label="Dec(deg)", width=1100)
fig.scatter(ra.hours, dec.degrees, fill_color=colors, line_color="red", size=7)
fig.line(ra.hours, dec.degrees, line_color="blue")
show(fig)

In [16]:
fig = figure(title="Moon standstills Dec, starting at " + t_start.tt_strftime(), x_axis_label="days", y_axis_label="Dec(deg)", width=1100)
fig.scatter(t_max_dec.tt - t_start.tt, dec.degrees, fill_color=colors, line_color="red", size=7)
fig.line(t_max_dec.tt - t_start.tt, dec.degrees, line_color="blue")
show(fig)

In [17]:
fig = figure(title="Moon standstills RA, starting at " + t_start.tt_strftime(), x_axis_label="days", y_axis_label="RA(hours)", width=1100)
fig.scatter(t_max_dec.tt - t_start.tt, ra.hours, fill_color=colors, line_color="red", size=7)
fig.line(t_max_dec.tt - t_start.tt, ra.hours, line_color="blue")
show(fig)

In [18]:
fig = figure(title="Moon standstills distance, starting at " + t_start.tt_strftime(), x_axis_label="days", y_axis_label="km", width=1100)
fig.scatter(t_max_dec.tt - t_start.tt, dist.km, fill_color=colors, line_color="red", size=7)
fig.line(t_max_dec.tt - t_start.tt, dist.km, line_color="blue")
show(fig)